https://www.kaggle.com/dansbecker/learning-to-use-xgboost

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
print(data.shape)
print(data.isnull().any()['SalePrice'])


(1460, 81)
False


In [2]:
data.dropna(axis=0, subset=['SalePrice'], inplace=True)
print(data.shape)


(1460, 81)


In [3]:
y = data.SalePrice
print(y.shape)
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])
X.head()

(1460,)


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


In [4]:
train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.fit_transform(test_X)

In [5]:
# ここで既にnp arrayになっている。train_test_splitがnarrayのみ対応
# だからそれ使うと以降はnarray
train_X

array([[ 591.        ,   60.        ,   64.        , ...,    0.        ,
           9.        , 2008.        ],
       [1149.        ,   50.        ,   70.72747497, ...,    0.        ,
           8.        , 2008.        ],
       [1112.        ,   60.        ,   80.        , ...,    0.        ,
           9.        , 2008.        ],
       ...,
       [ 226.        ,  160.        ,   21.        , ...,    0.        ,
           5.        , 2009.        ],
       [ 616.        ,   85.        ,   80.        , ...,    0.        ,
           5.        , 2010.        ],
       [ 259.        ,   60.        ,   80.        , ...,    0.        ,
           5.        , 2008.        ]])

In [24]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(train_X, train_y, verbose=False)

predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print('MAE: ', mean_absolute_error(predictions, test_y))


MAE:  15365.669809503424


In [31]:
# model turning
# number of model cycle
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=5, eval_set=[(test_X, test_y)],
            verbose=False)

predictions = my_model.predict(test_X)
print('MAE: ', mean_absolute_error(predictions, test_y))

MAE:  15420.980543664384


In [32]:
# learning rate
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(train_X, train_y, early_stopping_rounds=5, eval_set=[(test_X, test_y)],
            verbose=False)
predictions = my_model.predict(test_X)
print('MAE: ', mean_absolute_error(predictions, test_y))

MAE:  15397.622602739726


In [46]:
# learning rate
my_model = XGBRegressor(n_estimators=10000000, learning_rate=0.05, n_jobs=8)
my_model.fit(train_X, train_y, early_stopping_rounds=50000, eval_set=[(test_X, test_y)],
            verbose=False)
predictions = my_model.predict(test_X)
print('MAE: ', mean_absolute_error(predictions, test_y))

MAE:  15277.242551369864
